# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

## 1.Start by creating a new Notebook for this assignment


In [ ]:
#done

### To start we need pandas, numpy and Beautifulsoup

In [1]:
#step1. import standard libraries
import pandas as pd
import numpy as np
import requests

In [2]:
#step1.1. import bsp
!pip install beautifulsoup4
from bs4 import BeautifulSoup

You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Get PostalCodes from Wikipedia

In [3]:
# Step2. Get data from Wikipedia
wiki_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page= requests.get(wiki_link).text
wiki_soup = BeautifulSoup(wiki_page,'xml')


To create final dataframe we need import row data and clean up it:

In [5]:
#Step3. Get row data

#Step3.1: Get row data table
table = wiki_soup.find('table')

#initialize
Postcode      = []
Borough       = []
Neighbourhood = []

#Step 3.2: Clean row data data
for tr_cell in table.find_all('tr'):#Iterating through rows and columns
   

    var = 1
    PC     = -1
    Brgh = -1
    Nbh = -1
    
    for td_cell in tr_cell.find_all('td'): #Iterating through rows and columns
        if var == 1: 
            PC = td_cell.text
        if var == 2: 
            Brgh = td_cell.text
            checkBrgh = td_cell.find('a')
            
        if var == 3: 
            Nbh = str(td_cell.text).strip()
            checkNbh = td_cell.find('a')
            
        var +=1
        
#handling errors:   

 if (PC== 'Not assigned' or Brgh == 'Not assigned' or Nbh == 'Not assigned'): 
        continue
    try:
        if ((checkBrgh is None) or (checkNbh is None)):
            continue
    except:
        pass
    if(PC == -1 or Brgh == -1 or Nbh == -1):
        continue
        
    Postcode.append(PC)
    Borough.append(Brgh)
    Neighbourhood.append(Nbh)

In [6]:
#More than one neighborhood can exist in one postal code area.
#For example, in the table on the Wikipedia page, you will notice that M5A
#is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma as shown
#in row 11 in the above table.

#Step4. Get postal code with more than 1 neighbourhood

PC_unique = set(Postcode)

UniquePostcode     = []
UniqueBorough    = []
UniqueNeighbourhood = []

for pc_uniqueGD in PC_unique:
    pcv = ''; brgv = ''; nghv = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if pc_uniqueGD == postcode_element:
            pcv = postcode_element;
            brgv = Borough[postcode_idx]
            if nghv == '': 
                nghv = Neighbourhood[postcode_idx]
            else:
                nghv = nghv + ', ' + Neighbourhood[postcode_idx]
    UniquePostcode.append(pcv)
    UniqueBorough.append(brgv)
    UniqueNeighbourhood.append(nghv)

#Step5. Get clean pd.dataframe from row data from wikipedia
TableData= {'PostalCode': UniquePostcode, 'Borough': UniqueBorough, 'Neighbourhood': UniqueNeighbourhood}
df_TableData = pd.DataFrame.from_dict(TableData)
df_TableData.head(10)

,Borough,Neighbourhood,PostalCode
0,Downtown Toronto,Church and Wellesley,M4Y
1,Etobicoke,Islington Avenue,M9A
2,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",M5T
3,West Toronto,High Park,M6P
4,Scarborough,"Agincourt North, Milliken",M1V
5,North York,Maple Leaf Park,M6L
6,East York,Thorncliffe Park,M4H
7,North York,Bedford Park,M5M
8,North York,"Northwood Park, York University",M3J
9,Downtown Toronto,"Toronto Islands, Union Station",M5J


In [7]:
df_TableData.shape

(84, 3)

Thank you for grading,
Marija